In [1]:
#api key
API_KEY ='0d02265676b10a2702f45b78f36b0e1b0840bb05eb4c436ecf217440c2b80254'

In [2]:
#instantiate XBRL api
import pandas as pd
from sec_api import XbrlApi

xbrlApi = XbrlApi(API_KEY)

In [3]:
#instantiate Query api
from sec_api import QueryApi

queryApi = QueryApi(api_key=API_KEY)

In [18]:
#grab filing URL
import pandas as pd


def standardize_filing_url(url):
  return url.replace('ix?doc=/', '')


def get_10K_metadata(start_year = 2021, end_year = 2022):
  frames = []

  for year in range(start_year, end_year + 1):
    number_of_objects_downloaded = 0

    for month in range(1, 13):
      padded_month = str(month).zfill(2) # "1" -> "01"
      date_range_filter = f'filedAt:[{year}-{padded_month}-01 TO {year}-{padded_month}-31]'
      form_type_filter  = f'formType:"10-K" AND NOT formType:("10-K/A", NT)'
      lucene_query = date_range_filter + ' AND ' + form_type_filter

      query_from = 0
      query_size = 200

      while True:
        query = {
          "query": lucene_query,
          "from": query_from,
          "size": query_size,
          "sort": [{ "filedAt": { "order": "desc" } }]
        }

        response = queryApi.get_filings(query)
        filings = response['filings']

        if len(filings) == 0:
          break
        else:
          query_from += query_size

        metadata = list(map(lambda f: {'ticker': f['ticker'], 
                                       'cik': f['cik'], 
                                       'formType': f['formType'], 
                                       'filedAt': f['filedAt'], 
                                       'filingUrl': f['linkToFilingDetails']
                                      }, filings))

        df = pd.DataFrame.from_records(metadata)
        # remove all entries without a ticker symbol
        df = df[df['ticker'].str.len() > 0]
        df['filingUrl'] = df['filingUrl'].apply(standardize_filing_url)
        frames.append(df)
        number_of_objects_downloaded += len(df)

    print(f'✅ Downloaded {number_of_objects_downloaded} metadata objects for year {year}')

  result = pd.concat(frames)

  print(f'✅ Download completed. Metadata downloaded for {len(result)} filings.')

  return result

In [25]:
#download metadata
metadata_10K = get_10K_metadata(start_year=2017, end_year=2018)

✅ Downloaded 5695 metadata objects for year 2017
✅ Downloaded 5683 metadata objects for year 2018
✅ Download completed. Metadata downloaded for 11378 filings.


In [26]:
#save metadata
metadata_10K.to_pickle('metadata_10K 2017_2018.pkl')

In [4]:
#split download per year
import pandas as pd
year='2021'
data=pd.read_pickle(f'metadata_10K 2021_2022.pkl')
data[data.filedAt.astype(str).str[:4]==year].to_pickle(f'json {year}.pkl')


In [2]:
import pandas as pd

In [8]:
#fetch XBRL
import numpy as np
import pandas as pd
from tqdm import tqdm
import concurrent.futures
def fetch_xbrl_json(url):
    try:
        xbrl_json = xbrlApi.xbrl_to_json(htm_url=url)
        return xbrl_json
    except:
        return np.nan
json_list = []


data=pd.read_pickle('json 2021.pkl').reset_index()


# Using ThreadPoolExecutor for concurrent processing
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(fetch_xbrl_json, row.filingUrl) for index, row in data.iterrows()]
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
        json_list.append(future.result())
data['json'] = json_list
data.to_pickle('json_fetched_2021.pkl')

100%|██████████| 5997/5997 [13:42<00:00,  7.29it/s]  


In [10]:
data.head()

,index,ticker,cik,formType,filedAt,filingUrl,json
0,0,AMD,2488,10-K,2021-01-29T17:24:00-05:00,https://www.sec.gov/Archives/edgar/data/2488/0...,{'CoverPage': {'EntityCentralIndexKey': '00015...
1,2,ASTI,1350102,10-K,2021-01-29T16:02:57-05:00,https://www.sec.gov/Archives/edgar/data/135010...,"{'CoverPage': {'DocumentType': '10-K', 'Docume..."
2,3,DUSYF,1551887,10-K,2021-01-29T15:17:44-05:00,https://www.sec.gov/Archives/edgar/data/155188...,{'CoverPage': {'EntityRegistrantName': 'Digipa...
3,5,KSU,54480,10-K,2021-01-29T13:52:54-05:00,https://www.sec.gov/Archives/edgar/data/54480/...,{'CoverPage': {'EntityCentralIndexKey': '00002...
4,6,FREVS,36840,10-K,2021-01-29T13:29:05-05:00,https://www.sec.gov/Archives/edgar/data/36840/...,"{'CoverPage': {'DocumentType': '10-K', 'Docume..."


In [3]:
#loop for xbrl fetch
import numpy as np
import pandas as pd
from tqdm import tqdm
import concurrent.futures
# Function to fetch xbrl_json
def fetch_xbrl_json(url):
    try:
        xbrl_json = xbrlApi.xbrl_to_json(htm_url=url)
        return xbrl_json
    except:
        return np.nan
for year in np.arange(2017,2023):

    # List to store JSON results
    data=pd.read_pickle(f'json {year}.pkl').reset_index()
    json_list = []

    # Using ThreadPoolExecutor for concurrent processing
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(fetch_xbrl_json, row.filingUrl) for index, row in data.iterrows()]
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            json_list.append(future.result())

    data['json'] = json_list
    data.to_pickle(f'json_fetched_{year}.pkl')

100%|██████████| 5526/5526 [13:14<00:00,  6.95it/s]  


: 

In [1]:
#loop for partial tag match
import pandas as pd
from tqdm import tqdm
import numpy as np

def check_for_partial_matches(df, key_to_check, delimiter=';'):
    def find_partial_matches(json_data, key_to_check, parent_key=None):
        matches = set()
        if isinstance(json_data, dict):
            for key, value in json_data.items():
                current_key = key if parent_key is None else f"{parent_key}.{key}"
                if key_to_check.lower() in key.lower():
                    matches.add(parent_key if parent_key else key)
                if isinstance(value, dict):
                    matches.update(find_partial_matches(value, key_to_check, current_key))
                elif isinstance(value, list):
                    for item in value:
                        if isinstance(item, dict):
                            matches.update(find_partial_matches(item, key_to_check, current_key))
        return matches

    col_name_match = f'contains_{key_to_check.lower()}'
    col_name_key = f'key_contains_{key_to_check.lower()}'

    # Apply the function to each row and collect matches
    df[col_name_key] = df['json'].apply(lambda x: delimiter.join(find_partial_matches(x, key_to_check)))
    df[col_name_match] = df[col_name_key].apply(lambda x: len(x) > 0)

    return df
key_to_check = 'BusinessCombinationRecognizedIdentifiableAssets'


for year in tqdm(np.arange(2021,2022)):
    data=pd.read_pickle(f'json_fetched_{year}.pkl')
# Check for partial matches in the json column
    result_partial_matches_df = check_for_partial_matches(data, key_to_check)
    result_partial_matches_df[result_partial_matches_df['contains_businesscombinationrecognizedidentifiableassets']==True].to_pickle(f'checked partial/partial_check{year}.pkl')

100%|██████████| 1/1 [04:48<00:00, 288.35s/it]


In [2]:
#loop for expanding the content of the tag
import pandas as pd
from pandas import json_normalize
import numpy as np
from tqdm import tqdm

# Function to flatten the DataFrame
def flatten_dataframe(df):
    rows = []
    for _, row in df.iterrows():
        details = row['details']
        if isinstance(details, list):
            for item in details:
                if isinstance(item, dict):
                    new_row = {**row.to_dict(), **item}
                else:
                    new_row = {**row.to_dict(), 'details': item}
                rows.append(new_row)
        elif isinstance(details, dict):
            new_row = {**row.to_dict(), **details}
            rows.append(new_row)
        else:
            new_row = row.to_dict()
            rows.append(new_row)
    return pd.DataFrame(rows)

# Function to process segments
def process_segments(segment):
    try:
        flattened_segment = json_normalize(segment)
        combined_values = flattened_segment.apply(lambda x: ';'.join(x.astype(str)), axis=1)
        return ';'.join(combined_values)
    except:
        return np.nan

# Initialize an empty list to store the results

for year in np.arange(2021,2022):
    all_expanded_dfs = []
    input=pd.read_pickle(f'/Users/jonathantanone/Purchased-intangibles/checked partial/partial_check{year}.pkl')
    # Loop through each row of the DataFrame
    for idx, row in tqdm(input.iterrows()):
        try:
            # Extract the input data from the JSON column
            input_data = row['json'][row['key_contains_businesscombinationrecognizedidentifiableassets']]
            
            # Create a DataFrame from the input data
            df = pd.DataFrame(input_data.items(), columns=['category', 'details'])
            
            # Flatten the DataFrame
            flattened_df = flatten_dataframe(df)
            
            # Remove prefixes from the 'category' column
            prefixes = [
                'BusinessCombinationRecognizedIdentifiableAssetsAcquiredAndLiabilitiesAssumed',
                'BusinessCombinationRecognizedIdentifiable',
                'BusinessCombinationIdentifiable',
                'BusinessCombinationRecognized',
                'BusinessCombination',
            ]
            for prefix in prefixes:
                flattened_df['category'] = flattened_df['category'].str.replace(f'^{prefix}', '', regex=True)
            try:
                # Process segments and add the results to the DataFrame
                flattened_df['segment_result'] = flattened_df['segment'].apply(process_segments)
                
                # Split the 'segment_result' column into multiple columns
                segment_columns = flattened_df['segment_result'].str.split(';', expand=True)
                segment_columns = segment_columns.rename(lambda x: f'segment_{x+1}', axis=1)
                
                # Combine the new columns with the original DataFrame
                expanded_df = pd.concat([flattened_df, segment_columns], axis=1)
                
                # Drop the original 'segment_result' column if it's no longer needed
                expanded_df = expanded_df.drop(columns=['segment_result'])
            except:
                expanded_df = flattened_df
            # Append the expanded DataFrame to the list
            all_expanded_dfs.append(expanded_df)
        except:
            all_expanded_dfs.append(pd.DataFrame(columns=['check']))

    # Concatenate all the expanded DataFrames
    input['table'] = all_expanded_dfs
    input.iloc[:,1:].reset_index().iloc[:,1:].to_pickle(f'PPA tabulated/data_{year}.pkl')



1645it [00:06, 261.65it/s]


In [3]:
#loop for expanding the content
import pandas as pd
import numpy as np

def expand_table_skip_empty(df):
    expanded_df_list = []
    for index, row in df.iterrows():
        table_df = row['table']
        if not table_df.empty:
            table_df['ticker'] = row['ticker']
            table_df['cik'] = row['cik']
            table_df['filedAt'] = row['filedAt']
            expanded_df_list.append(table_df)
    
    if expanded_df_list:
        expanded_df = pd.concat(expanded_df_list, ignore_index=True)
    else:
        expanded_df = pd.DataFrame()  # Return an empty DataFrame if all tables are empty
    return expanded_df

for a in np.arange(2021,2022):
    data=pd.read_pickle(f'PPA tabulated/data_{a}.pkl')
    expanded_df=expand_table_skip_empty(data)
    expanded_df.to_csv(f'expanded_df/ppa_expanded_{a}.csv')


In [5]:
data=pd.read_csv('expanded_df/ppa_expanded_2017.csv')

/var/folders/00/q80vtjnd0bv_yp6wck36g7w40000gn/T/ipykernel_4269/3514968346.py:1: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('expanded_df/ppa_expanded_2017.csv')
